In [1]:
# リスト 5.3.1 初期処理と変数宣言（説明コメント付き）
# ------------------------------------------------------------
# 目的:
# - Mac（特に Apple Silicon / MKL 由来の OpenMP ランタイム）環境で発生しやすい
#   "KMP_DUPLICATE_LIB_OK" 警告クラッシュの回避設定
# - ノートブック/学習ログの見通しを良くするための警告抑止（デモ用）
# - Word2Vec 埋め込み次元数および LSTM に入力する最大系列長の基準値を宣言
#
# 注意:
# - 以下の環境変数・警告抑止は「実験用の便宜措置」です。プロダクションでは
#   依存ライブラリの統一や仮想環境管理（conda/venv/uv など）で根本対応するのが望ましい。
# - EMBEDDING_DIM / MAX_LEN は下流のモデル定義・前処理（Tokenizer, padding 戦略）
#   と一貫している必要があります（不一致だと shape エラーや性能劣化につながる）。
# ------------------------------------------------------------

# --- Mac の OpenMP ランタイム多重ロード問題の暫定回避 ---------------------------------
# Intel MKL / libomp 等が複数ロードされるとクラッシュするケースがあり、Mac で学習時に
# "KMP_DUPLICATE_LIB_OK" を True にして回避することがある。
# 本来は「ライブラリの競合解消（単一の OpenMP に統一）」が理想。恒久対策ではない点に注意。
import os
import platform

if platform.system() == "Darwin":
    os.environ["KMP_DUPLICATE_LIB_OK"] = "True"  # デモ/学習用の一時対応

# --- 警告抑止（デモの可読性向上のため） ---------------------------------------------------
# 学習途中の FutureWarning / DeprecationWarning などで出力がノイズ化するのを防ぐ。
# ただし、重要な警告まで隠すリスクがあるため、本番/検証では必要な種類だけを個別に filter すること。
import warnings

warnings.filterwarnings("ignore")

# --- 埋め込み次元（Word2Vec の隠れ層ノード数 = ベクトル次元数） ---------------------------
# 一般的な選択肢: 100〜300（コーパス規模が小〜中の場合）。大きくするほど表現力は上がるが、
# 学習時間・メモリ（O(|V|×D)）が増加し過学習のリスクも高まる。
# 既存の学習済みモデル（例: fastText 300 次元）と互換を取りたい場合は、その次元に合わせる。
EMBEDDING_DIM = 300

# --- LSTM に入力する最大系列長（トークン数） ----------------------------------------------
# 前処理側（Tokenizer/分かち書き）で得られた系列を padding/truncation する長さ。
# 値を大きくすると長距離依存を捉えやすくなるが、計算負荷とメモリ消費が増える。
# 日本語文で 50 はやや短め。短文中心のデータ（ツイート等）には妥当だが、長文なら 128〜256 も検討。
# 下流モデル（Embedding 層の input_length, LSTM の time_steps）や学習用 DataLoader と整合させること。
MAX_LEN = 50

# --- 参考: 再現性確保のための乱数シード（必要に応じて使用） ------------------------------
# import random, numpy as np
# import torch
# SEED = 42
# random.seed(SEED); np.random.seed(SEED)
# torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
# torch.backends.cudnn.deterministic = True; torch.backends.cudnn.benchmark = False
#
# --- 参考: gensim 4.x で学習する場合のパラメータ整合 --------------------------------------
# - Word2Vec(vector_size=EMBEDDING_DIM, ...)  # ※ gensim 3 系の size は 4 系では vector_size に名称変更
# - 学習済みベクトルを使う場合は EMBEDDING_DIM と一致していないとロード時に shape 不整合となる。

In [2]:
# リスト5.3.2 テキスト取得（説明コメント付き）
# ------------------------------------------------------------
# 目的:
# - Wikipedia から「歴史」「地理」「テスト」カテゴリの見出し語に対する
#   日本語サマリーを取得し、後続の前処理や学習（例: 文書分類/系列モデル）で使える
#   文字列配列に整形する。
#
# 設計メモ:
# - wikipedia ライブラリの summary/page 取得は、曖昧さ回避（Disambiguation）や
#   ページ未存在（PageError）で例外を投げることがあるため、実運用では try/except を推奨。
# - 取得結果は「文字列」の配列（list[str]）。後段でトークナイズ→語彙化→パディング等を行う。
# - API 呼び出しは外部リソースに依存するため、ネットワーク断やレート制限も考慮が必要。
#   デモでは単純な再試行やタイムスリープは省略している。
# ------------------------------------------------------------

import wikipedia
from wikipedia.exceptions import PageError, DisambiguationError

# --- Wikipedia を日本語に設定 -------------------------------------------------------------
# これにより page/summary の検索対象が日本語版 Wikipedia になる。
wikipedia.set_lang("ja")

# --- 学習データ(歴史)のタイトル ----------------------------------------------------------
list1 = [
    "大和時代",
    "奈良時代",
    "平安時代",
    "鎌倉時代",
    "室町時代",
    "安土桃山時代",
    "江戸時代",
    "藤原道長",
    "平清盛",
    "源頼朝",
    "北条早雲",
    "伊達政宗",
    "徳川家康",
    "武田信玄",
    "上杉謙信",
    "今川義元",
    "毛利元就",
    "足利尊氏",
    "足利義満",
    "北条泰時",
]

# --- 学習データ(地理)のタイトル ----------------------------------------------------------
list2 = [
    "東北地方",
    "関東地方",
    "中部地方",
    "近畿地方",
    "中国地方",
    "四国地方",
    "九州地方",
    "北海道",
    "秋田県",
    "福島県",
    "宮城県",
    "新潟県",
    "長野県",
    "山梨県",
    "静岡県",
    "愛知県",
    "栃木県",
    "群馬県",
    "千葉県",
    "神奈川県",
]

# --- テストデータのタイトル --------------------------------------------------------------
list3 = ["織田信長", "豊臣秀吉", "青森県", "北海道"]


# --- 取得用の安全ラッパ ---------------------------------------------------------------
# Wikipedia API は以下の例外が発生しうる:
# - DisambiguationError: 曖昧さ回避ページにぶつかった場合
# - PageError: ページが存在しない場合
# この関数では例外を握りつぶさず、最小限のフォールバック文字列を返す。
def safe_summary(title: str, sentences: int | None = None) -> str:
    """
    指定タイトルの Wikipedia サマリーを返す。
    失敗時はエラー内容を含む短い代替テキストを返す。
    """
    try:
        # sentences を指定すると要約長を短くできる（None ならライブラリ既定）
        return wikipedia.summary(title, sentences=sentences)
    except DisambiguationError as e:
        # 曖昧さ回避: 候補の一部を添えて知らせる（学習に使う場合は除外も選択肢）
        few = ", ".join(e.options[:3])
        return f"[Disambiguation] '{title}' は曖昧です。例: {few} ..."
    except PageError:
        return f"[PageError] '{title}' に対応するページが見つかりません。"
    except Exception as ex:
        # ネットワーク等のその他エラー
        return f"[Error] '{title}' の取得に失敗しました: {type(ex).__name__}: {ex}"


# --- 各リストのタイトルからサマリー文字列配列を作成 --------------------------------------
# 大規模取得では API 呼び出し回数が多くなるため、必要に応じて
# ・キャッシュ（ローカル保存）
# ・バックオフ（time.sleep）
# ・並列化（ただし Wikipedia API の規約/負荷に留意）
# などを検討する。
list1_w = [safe_summary(item) for item in list1]
list2_w = [safe_summary(item) for item in list2]
list3_w = [safe_summary(item) for item in list3]

# --- すべての取得結果を 1 つのリストに集約 ------------------------------------------------
# 下流で学習/評価に使う「コーパス」。メタ情報（ラベル: 歴史/地理/テスト）を
# 併せて管理したい場合は、別途同長のラベル配列や (text, label) のタプル配列にする。
list_all_w = list1_w + list2_w + list3_w

# --- 動作確認（任意） -------------------------------------------------------------------
# 先頭 1–2 件を軽く出力して取得できているかを確認したい場合:
# print(list_all_w[0][:120], "...")
# print(list_all_w[len(list1_w)][:120], "...")

In [3]:
# リスト 5.3.3 テキストに対して単語毎にブランクを入れる（説明コメント付き）
# ------------------------------------------------------------
# 目的:
# - 直前で収集した Wikipedia サマリー（list1_w, list2_w, list3_w）を
#   形態素解析して「分かち書き」（語と語の間を半角スペースで区切る）に変換する。
# - 後段の処理（例: Word2Vec / Doc2Vec / BoW / TF-IDF / RNN/LSTM など）で、
#   トークナイザ不要のシンプルな「スペース区切りトークン列」を入力として扱えるようにする。
#
# 設計メモ:
# - 日本語は英語と違い空白を単語境界に用いないため、多くのモデル/ベクトル化器に入力する前に
#   形態素解析で単語境界を明示する必要がある。
# - 本コードは Janome を採用（純 Python で導入容易・辞書同梱）。高速性やドメイン適合度が
#   重要なら MeCab(+IPA/NEologd) 等の代替も検討する。
# - wakati=True は「表層形の列」を返すため、品詞情報は捨てて単純な単語列にする運用。
#   品詞に応じたフィルタ（名詞/動詞の原形のみ等）を行いたい場合は Token の属性を使う別関数を用意する。
# ------------------------------------------------------------

from janome.tokenizer import Tokenizer

# Tokenizer の生成は比較的コストがかかるため、関数外で 1 度だけインスタンス化して再利用する。
# （大量文書を処理する場合のパフォーマンス観点）
t = Tokenizer()


def wakati(text: str) -> str:
    """
    文字列 `text` を Janome で分かち書きし、半角スペース区切りで返す。

    パラメータ:
        text (str): 日本語テキスト（Wikipedia サマリーなど）
    戻り値:
        str: "単語1 単語2 単語3 ..." のようなスペース区切り列
    備考:
        - wakati=True を指定しているため、Token オブジェクトではなく表層形のイテラブルが得られる。
        - 記号・数字・助詞なども出力に含まれる（Janome の標準挙動）。
          下流で除去/正規化したい場合は別途前処理（例: 正規化、品詞フィルタ）を追加する。
        - 未知語（固有名詞・新語）は辞書次第で分割精度が変わる点に留意。
    """
    # Tokenizer.tokenize(..., wakati=True) は「表層形のイテレータ」を返す
    # 例: "徳川家康 は 江戸 幕府 を 開い た" → ' ' で join して 1 本の文字列に
    words_iter = t.tokenize(text, wakati=True)
    return " ".join(words_iter)


# --- 分かち書きの実行 ---------------------------------------------------------
# list*_w は直前ステップ（リスト 5.3.2）で取得した Wikipedia サマリー配列を想定
# * _w: raw 文（sentence/string）のリスト
# * _x: 分かち書き（tokenized）後の文のリスト
list1_x = [wakati(w) for w in list1_w]  # 歴史カテゴリ
list2_x = [wakati(w) for w in list2_w]  # 地理カテゴリ
list3_x = [wakati(w) for w in list3_w]  # テスト用（評価/可視化など）

# 学習・評価で一括処理したいケースに備えてマージ
list_all_x = list1_x + list2_x + list3_x

# --- 参考: 以降の一般的な流れ（ここでは実装しない） --------------------------
# 1) 正規化（NFKC）、英数字の統一、記号除去など（必要に応じて）
# 2) 語彙化（Vocabulary 構築）→ 単語ID列に変換
# 3) 固定長化（MAX_LEN へのパディング/トランケーション）
# 4) 埋め込み層 or 事前学習ベクトルの読み込み（Word2Vec/BERT トークナイザ等）
# 5) 学習（分類/系列ラベリング/類似度計算など）

In [7]:
# リスト 5.3.4 学習データ作成（説明コメント付き）

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenizer:
# - テキストを語彙辞書（単語→整数ID）に変換し、整数ID列（シーケンス）へ数値化するユーティリティ。
# - 既定では「出現頻度の高い順」に 1, 2, 3, ... の ID が割り当てられる（0 はパディング用に予約しない設計）。
# - 実務では oov_token（未知語トークン）を指定しておくと、学習時に未観測な語にも頑健になる。
tokenizer = Tokenizer()

# 学習・検証で使う全テキストを引数にして辞書を作成する
# 注意（データリーク）:
# - ここでは list_all_x（学習+検証+テスト相当）をまとめて fit しているため、
#   厳密には検証・テスト側の語彙情報が学習側に漏れる（軽微だが情報リーク）。
# - 再現実験や教材としては簡潔だが、厳密評価では「学習コーパスのみで fit」→
#   「検証/テストは texts_to_sequences のみ」を推奨。
tokenizer.fit_on_texts(list_all_x)

# 単語一覧（語→ID の辞書）を取得
# 例: {'江戸時代': 1, 'に': 2, 'は': 3, ...}
word_index = tokenizer.word_index

# 総単語数（ユニーク語彙数）を取得
num_words = len(word_index)
print("総単語数: ", num_words)

# 変換前の検証用テキスト確認（分かち書き済みの文字列）
# - list3_x はテスト用タイトルの Wikipedia サマリーを分かち書きした配列（前工程参照）。
print("変換前テキスト: ", list3_x[0])

# テキストの数値化:
# - texts_to_sequences は、各テキストを語彙辞書に基づいて「整数IDのリスト」へ変換する。
# - 未知語は既定では無視され、ID 化されない（→ シーケンスから脱落）。
#   未知語を保持したい場合は Tokenizer(oov_token='[UNK]') などを利用。
sequence_test = tokenizer.texts_to_sequences(list3_x)

# 変換結果確認（例: [12, 345, 78, ...]）
print("変換後: ", sequence_test[0])

# 単語のパディング:
# - ニューラルネットに固定長テンソルを与えるため、pad_sequences で
#   「短い文は 0 で埋める」「長い文は途中で切り詰める」処理を行う。
# - 既定では先頭側にパディング（padding='pre'）、長い場合は先頭側を切る（truncating='pre'）。
#   LSTM の方向やモデル設計に応じて 'post' に変えることも多い。
sequence_test = pad_sequences(sequence_test, maxlen=MAX_LEN)

# パディング後の確認（長さが MAX_LEN の固定長ベクトルに）
print("パディング後: ", sequence_test[0])

# 学習データ（歴史: list1_x、地理: list2_x）に対しても同じ数値化→パディングを適用
# - fit はすでに済んでいるため、ここでは texts_to_sequences + pad_sequences のみ。
sequence_train = tokenizer.texts_to_sequences(list1_x + list2_x)
sequence_train = pad_sequences(sequence_train, maxlen=MAX_LEN)

# 正解ラベルの作成:
# - 2 クラス分類を想定し、歴史カテゴリを 0、地理カテゴリを 1 として付与。
# - 学習用 y は学習データの件数に合わせて 0/1 を連結。
# - 検証/テスト用 y は list3_x の内容（例: ['織田信長','豊臣秀吉','青森県','北海道']）に合わせて
#   先頭 2 件を歴史=0、後ろ 2 件を地理=1 として作成（デモ簡略化のための固定割当）。
Y_train = np.array([0] * len(list1_x) + [1] * len(list2_x))
Y_test = np.array([0] * 2 + [1] * 2)

# ラベル配列の確認
print("正解データ(学習用): ", Y_train)
print("正解データ(検証用): ", Y_test)

# ---------------------------------------------
# 補足（実務上のベストプラクティス）:
# - 語彙構築は学習データのみに限定（データリーク回避）。
# - トークン頻度に基づく語彙上限 num_words を設定し、希少語をまとめて OOV へ。
# - pad_sequences の padding/truncating はモデル特性に合わせて 'post' を検討。
# - ラベルは One-Hot ではなく整数ラベルのままでも SparseCategoricalCrossentropy で学習可。
# - クラス不均衡がある場合は class_weight やサンプリングで補正。
# ---------------------------------------------

総単語数:  1465
変換前テキスト:  織田   信長 （ おだ   のぶ な が ） は 、 日本 の 戦国 時代 から 安土 桃山 時代 にかけて の 武将 ・ 大名 。 戦国 の 三 英傑 の 一 人 。 
 尾張 国 （ 現在 の 愛知 県 ） 出身 。 織田 信秀 の 嫡男 。 家督 争い の 混乱 を 収め た 後 に 、 桶 狭間 の 戦い で 今川 義元 を 討ち取り 、 勢力 を 拡大 し た 。 足利 義昭 を 奉じ て 上洛 し 、 後 に は 義昭 を 追放 する こと で 、 畿内 を 中心 に 独自 の 中央 政権 （ 「 織田 政権 」 ） を 確立 し て 天下 人 と なっ た 。 しかし 、 天正 10 年 6 月 2 日 （ 1582 年 6 月 21 日 ） 、 家臣 ・ 明智 光秀 に 謀反 を 起こさ れ 、 本能寺 で 自害 し た 。 
 これ まで 信長 の 政権 は 、 豊臣 秀吉 による 豊臣 政権 、 徳川 家康 が 開い た 江戸 幕府 へ の 流れ を つくっ た 画期的 な もの で 、 その 政治 手法 も 革新 的 な もの で ある と みなさ れ て き た 。 しかし 、 近年 の 歴史 学界 で は その 政策 の 前 時代 性 が 指摘 さ れる よう に なり 、 しばしば 「 中世 社会 の 最終 段階 」 と も 評さ れ 、 その 革新 性 を 否定 する 研究 が 主流 と なっ て いる 。
変換後:  [62, 103, 8, 459, 333, 30, 14, 7, 4, 2, 23, 1, 51, 10, 33, 54, 32, 10, 553, 1, 76, 13, 56, 3, 51, 1, 105, 644, 1, 161, 95, 3, 356, 36, 8, 351, 1, 205, 15, 7, 1423, 3, 62, 1424, 1, 320, 3, 558, 1425, 1, 1426, 6, 1427, 11, 75, 5, 2, 564, 565, 1, 342, 9, 204, 353, 6, 1428, 2, 359, 6, 254, 19, 11, 3, 72, 645, 6, 1429, 16, 1430, 19, 2, 75, 5, 4,